In [1]:
import json

In [2]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch("http://elasticsearch:9200")

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [8]:
index_name ="course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [11]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [32]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response =  es_client.search(index=index_name, body=search_query)
    result_docs=[]
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    
    return result_docs

In [34]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Do NOT use any external information or make assumptions.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    
    CONTEXT START
    ---------
    {context}
    ---------
    CONTEXT END
    
    """.strip()
    
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt 

In [36]:
from dotenv import load_dotenv
load_dotenv()

import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [37]:
def llm_response(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gemma2-9b-it",
        temperature=0.2,
    )
    return response.choices[0].message.content

In [38]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_response(prompt)
    return answer

In [41]:
query = "The course has already started, can I still enroll it?"
answer = rag(query)

In [42]:
answer

"Yes, even if you don't register, you're still eligible to submit the homeworks. \n"